**Socket**是网络编程的一个抽象概念。通常我们用一个Socket表示“打开了一个**网络链接**”，而打开一个Socket需要知道目标计算机的**IP地址**和**端口号**，再指定**协议类型**即可。


客户端
---
大多数连接都是**可靠的TCP连接**。创建TCP连接时，主动发起连接的叫**客户端**，被动响应连接的叫**服务器**。

举个例子，当我们在浏览器中访问新浪时，我们自己的计算机就是客户端，浏览器会主动向新浪的服务器发起连接。如果一切顺利，新浪的服务器接受了我们的连接，一个TCP连接就建立起来的，后面的通信就是发送网页内容了。

In [1]:
'''创建一个基于TCP连接的Socket'''
# 导入socket库:
import socket

# 创建一个socket:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 建立连接: 注意参数是一个tuple，包含地址和端口号。
s.connect(('www.sina.com.cn', 80))

创建一个Socket对象参数(还没有建立连接)：
1. AF_INET：IPv4协议
2. AF_INET6：IPv6协议
3. SOCK_STREAM：指定使用面向流的TCP协议，

客户端要主动发起TCP连接，必须知道服务器的IP地址和端口号。新浪网站的IP地址可以用域名 www.sina.com.cn 自动转换到IP地址，但是怎么知道新浪服务器的端口号呢？

答案是作为服务器，提供什么样的服务，端口号就必须固定下来。由于我们想要访问网页，因此新浪提供网页服务的服务器必须把端口号固定在80端口，因为**80端口**是**Web服务**的标准端口。其他服务都有对应的标准端口号，例如**SMTP服务**是**25端口**，**FTP服务**是**21端口**，等等。端口号小于1024的是Internet标准服务的端口，端口号大于1024的，可以任意使用。

建立TCP连接后，就可以向新浪服务器发送请求，要求返回首页的内容：

In [2]:
# 发送数据:
s.send(b'GET / HTTP/1.1\r\nHost: www.sina.com.cn\r\nConnection: close\r\n\r\n')

60

TCP连接创建的是**双向通道**，双方都可以同时给对方发数据。但是谁先发谁后发，怎么协调，要根据具体的协议来决定。例如，HTTP协议规定客户端必须先发请求给服务器，服务器收到后才发数据给客户端。

发送的文本格式必须符合HTTP标准，如果格式没问题，接下来就可以接收新浪服务器返回的数据了。  
**接收数据**时，调用**recv(max)**方法，一次最多接收指定的字节数，因此，在一个while循环中反复接收，直到recv()返回空数据，表示接收完毕，退出循环。

In [3]:
# 接收数据:
buffer = []
while True:
    # 每次最多接收1k字节:
    d = s.recv(1024)
    if d:
        buffer.append(d)
    else:
        break
data = b''.join(buffer)

当我们接收完数据后，调用close()方法关闭Socket，这样，一次完整的网络通信就结束了。

In [5]:
s.close()

接收到的数据包括HTTP头和网页本身，我们只需要把HTTP头和网页分离一下，把HTTP头打印出来，网页内容保存到文件。

In [ ]:
header, html = data.split(b'\r\n\r\n', 1)
print(header.decode('utf-8'))
# 把接收的数据写入文件:
with open('sina.html', 'wb') as f:
    f.write(html)

服务器
---
服务器进程首先要绑定一个**端口**并**监听**来自其他客户端的连接。如果某个客户端连接过来了，服务器就与该客户端建立Socket连接，随后的通信就靠这个Socket连接了。

所以，服务器会打开固定端口（比如80）监听，每来一个客户端连接，就创建该Socket连接。由于服务器会有大量来自客户端的连接，所以，服务器要能够区分一个Socket连接是和哪个客户端绑定的。一个Socket依赖4项来唯一确定一个Socket：
1. 服务器地址、
2. 服务器端口、
3. 客户端地址、
4. 客户端端口


但是服务器还需要同时响应多个客户端的请求，所以，每个连接都需要一个新的**进程**或者新的**线程**来处理，否则，服务器一次就只能服务一个客户端了。

一个简单的服务器程序：接收客户端连接，把客户端发过来的字符串加上Hello再发回去。
1. 首先，创建一个基于IPv4和TCP协议的Socket。
2. 然后，要绑定监听的地址和端口。服务器可能有多块网卡，可以绑定到某一块网卡的IP地址上，也可以用0.0.0.0绑定到所有的网络地址，还可以用127.0.0.1绑定到本机地址。127.0.0.1是一个特殊的IP地址，表示本机地址，如果绑定到这个地址，客户端必须同时在本机运行才能连接，也就是说，外部的计算机无法连接进来。端口号需要预先指定。因为我们写的这个服务不是标准服务，所以用9999这个端口号。请注意，小于1024的端口号必须要有管理员权限才能绑定。
3. 调用listen()方法开始监听端口，传入的参数指定等待连接的最大数量。
4. 服务器程序通过一个永久循环来接受来自客户端的连接，accept()会等待并返回一个客户端的连接。每个连接都必须创建新线程（或进程）来处理，否则，单线程在处理连接的过程中，无法接受其他客户端的连接。

In [ ]:
'''服务器程序'''
import socket,threading,time

# 创建一个socket:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 绑定监听端口:
s.bind(('127.0.0.1', 9999))

# 开始监听端口
s.listen(5)
print('Waiting for connection...')


    
def tcplink(sock, addr):
    print('Accept new connection from %s:%s...' % addr)
    sock.send(b'Welcome!')
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if not data or data.decode('utf-8') == 'exit':
            break
        sock.send(('Hello, %s!' % data.decode('utf-8')).encode('utf-8'))
    sock.close()
    print('Connection from %s:%s closed.' % addr)

while True:
    # 接受一个新连接:
    sock, addr = s.accept()
    # 创建新线程来处理TCP连接:
    t = threading.Thread(target=tcplink, args=(sock, addr))
    t.start()


In [ ]:
'''客户端程序'''
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 建立连接:
s.connect(('127.0.0.1', 9999))
# 接收欢迎消息:
print(s.recv(1024).decode('utf-8'))
for data in [b'Michael', b'Tracy', b'Sarah']:
    # 发送数据:
    s.send(data)
    print(s.recv(1024).decode('utf-8'))
s.send(b'exit')
s.close()

小结
---
用TCP协议进行Socket编程在Python中十分简单，对于客户端，要主动连接服务器的IP和指定端口，对于服务器，要首先监听指定端口，然后，对每一个新的连接，创建一个线程或进程来处理。通常，服务器程序会无限运行下去。

同一个端口，被一个Socket绑定了以后，就不能被别的Socket绑定了。